In [ ]:
import moth
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy


PATH = '/storage/home/adz6/group/project'
VNADATAPATH = os.path.join(PATH, 'vna_data')
MOTHPATH = os.path.join(PATH, 'moth')
MOTHDATA = os.path.join(MOTHPATH, 'data/datasets')


In [ ]:
moth_dataset_name = '210701_cres1_antenna_pattern_calibration.h5'
cres1_data = moth.data.MothData(os.path.join(MOTHDATA, moth_dataset_name))

In [ ]:
frequencies[3000]

In [ ]:
frequencies = np.int64(list(cres1_data.data.keys()))

frequency = frequencies[500]
radius = 10.0

dataframe = cres1_data.data[frequency]

pattern_data = dataframe[dataframe['r'] == radius]


In [ ]:
theta_vec = np.sort(pattern_data['theta'].unique())
z_vec = np.sort(pattern_data['z'].unique())

theta_grid, z_grid = np.meshgrid(theta_vec, z_vec)

s_grid = np.zeros(theta_grid.shape, dtype=np.complex128)

for i in range(theta_grid.shape[0]):
    for j in range(theta_grid.shape[1]):
        try:
            s_grid[i, j] = pattern_data[(pattern_data['theta'] == theta_grid[i, j]) & (pattern_data['z'] == z_grid[i, j])]['s-param'].item()
        except ValueError:
            print(f'({theta_grid[i, j]}, {z_grid[i, j]}) is NaN')
            s_grid[i, j] = np.nan
            
s_mag_grid = 10 * np.log10(abs(s_grid))
s_phase_grid = np.angle(s_grid)

theta_grid = np.radians(theta_grid)

s_mag_grid[np.argwhere(np.isnan(s_mag_grid))[:, 0], np.argwhere(np.isnan(s_mag_grid))[:, 1]] = np.mean(
    s_mag_grid[np.argwhere(~np.isnan(s_mag_grid))[:, 0], np.argwhere(~np.isnan(s_mag_grid))[:, 1]])

s_phase_grid[np.argwhere(np.isnan(s_phase_grid))[:, 0], np.argwhere(np.isnan(s_phase_grid))[:, 1]] = 0

s_phase_grid = np.unwrap(s_phase_grid)
s_phase_grid -= np.mean(s_phase_grid[:, 0])

In [ ]:
fig = plt.figure(figsize=(13, 8))
sns.set_theme(context='talk', style='ticks')
cmap = sns.color_palette(palette='mako', as_cmap=True)

#print(cmap)
ax = fig.add_subplot(1,1,1, projection='3d')
ax.view_init(30, -60)

#ax.plot3D(x,y,z, '.')
plot = ax.plot_surface(theta_grid, z_grid, s_mag_grid, cmap=cmap )

ax.set_zlim(-50, -10)

ax.set_ylabel('Axial Position from PCB (cm)', labelpad=16)
ax.set_xlabel('Angle (rad)', labelpad=10)
ax.set_zlabel('Calibrated\n S-Param Magnitude (dB)', labelpad=16)

plot.set_linewidth(.1)

plt.tight_layout()

fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1, projection='3d')
ax.view_init(0, 270)

#ax.plot3D(x,y,z, '.')
plot = ax.plot_surface(theta_grid, z_grid, s_mag_grid, cmap=cmap )

#ax.set_ylabel('Axial Position from PCB (cm)', labelpad=16)
ax.set_xlabel('Angle (rad)', labelpad=10)
ax.set_zlabel('Calibrated\n S-Param Magnitude (dB)', labelpad=32)
ax.tick_params(axis = 'z', pad = 10)
plot.set_linewidth(.1)


ax.set_zlim(-50, -10)
plt.tight_layout()

fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1, projection='3d')
ax.view_init(90, 270)

#ax.plot3D(x,y,z, '.')
plot = ax.plot_surface(theta_grid, z_grid, s_mag_grid, cmap=cmap )

ax.set_ylabel('Axial Position from PCB (cm)', labelpad=36)
ax.set_xlabel('Angle (rad)', labelpad=16)
#ax.set_zlabel('Calibrated\n S-Param Magnitude (dB)', labelpad=16)
ax.tick_params(pad = 14)
plot.set_linewidth(.1)


ax.set_zlim(-50, -10)
plt.tight_layout()

In [ ]:
ax.tick_params

In [ ]:
fig = plt.figure(figsize=(13, 8))
cmap = sns.color_palette(palette='mako', as_cmap=True)
#print(cmap)
ax = fig.add_subplot(1,1,1, projection='3d')
ax.view_init(30, -60)

#ax.plot3D(x,y,z, '.')
ax.plot_surface(theta_grid, z_grid, s_phase_grid, cmap=cmap )

ax.set_zlim(0, 2*np.pi)
plt.tight_layout()

fig = plt.figure(figsize=(13, 8))
cmap = sns.color_palette(palette='mako', as_cmap=True)
#print(cmap)
ax = fig.add_subplot(1,1,1, projection='3d')
ax.view_init(0, -90)
ax.set_zlim(0, 2*np.pi)


#ax.plot3D(x,y,z, '.')
ax.plot_surface(theta_grid, z_grid, s_phase_grid, cmap=cmap )

#ax.set_zlim(-50, -10)
plt.tight_layout()

fig = plt.figure(figsize=(13, 8))
cmap = sns.color_palette(palette='mako', as_cmap=True)
#print(cmap)
ax = fig.add_subplot(1,1,1, projection='3d')
ax.view_init(90, 270)

#ax.plot3D(x,y,z, '.')
ax.plot_surface(theta_grid, z_grid, s_phase_grid, cmap=cmap )

#ax.set_zlim(-50, -10)
plt.tight_layout()

In [ ]:
spline_interpolation = scipy.interpolate.RectBivariateSpline(theta_vec, z_vec, s_mag_grid.T)

interp_theta = np.radians(np.linspace(0, 360, 201))
interp_z  = np.linspace(-1, 1, 201)

interp_s = spline_interpolation.__call__(interp_theta, interp_z)
print(interp_s)

In [ ]:
interp_grid_theta, interp_grid_z = np.meshgrid(interp_theta, interp_z)

fig = plt.figure(figsize=(13, 8))
cmap = sns.color_palette()
#print(cmap)
ax = fig.add_subplot(1,1,1, projection='3d')

#ax.plot3D(x,y,z, '.')
ax.plot_surface(interp_grid_theta, interp_grid_z, interp_s, color = cmap[0] )

ax.set_zlim(-50, -10)


In [ ]:
theta_vec = np.sort(pattern_data['theta'].unique())
z_vec = np.sort(pattern_data['z'].unique())

theta_grid, z_grid, s_grid = np.zeros((theta_vec.size, z_vec.size)), np.zeros((theta_vec.size, z_vec.size)), np.zeros((theta_vec.size, z_vec.size), dtype=np.complex128)

for i, theta in enumerate(theta_vec):
    for j, z in enumerate(z_vec):
        theta_grid[i, j] = theta
        z_grid[i, j] = z
        #print(theta, z, pattern_data[(pattern_data['theta'] == theta) & (pattern_data['z'] == z)]['s-param'])
        try:
            s_grid[i, j] = pattern_data[(pattern_data['theta'] == theta) & (pattern_data['z'] == z)]['s-param'].item()
        except ValueError:
            print(f'({theta}, {z}) is NaN')
            s_grid[i, j] = np.nan
            
s_grid = 10 * np.log10(abs(s_grid)) # convert to dB

x_grid = s_grid * np.cos(theta_grid)
y_grid = s_grid * np.sin(theta_grid)

In [ ]:
r = np.array(10 * np.log10(abs(pattern_data['s-param']))) # the value

theta = np.array(np.radians(pattern_data['theta']))
z = np.array(pattern_data['z'])

points = np.array([theta, z]).T
print(points.shape)

plot_z = np.linspace(-1, 1, 15)
plot_theta = np.radians(np.arange(0, 360, 1))


In [ ]:
z_grid, theta_grid = np.meshgrid(z, theta)

r_grid = r.reshape(r.size, 1).repeat(r.size, axis=1)

x_grid = r_grid * np.cos(theta_grid)
y_grid = r_grid * np.sin(theta_grid)

In [ ]:
print(theta_grid[:, 0])
print(r_grid[:, 0])
print(z_grid[0, :])

In [ ]:
fig = plt.figure()

ax = fig.add_subplot(1,1,1, projection='3d')

#x.plot3D(x,y,z, '.')
ax.plot_surface(x_grid, y_grid, z_grid)


In [ ]:
scipy.interpolate.griddata(points, r, (grid_points[:, 0], grid_points[:, 1]))

In [ ]:
fig = plt.figure()

ax = fig.add_subplot(1,1,1, projection='3d')

#x.plot3D(x,y,z, '.')
ax.plot_surface(xgrid[0, :], ygrid[0, :], zgrid[0, :])

#ax.set_xlim(-1e-2, 1e-2)
#ax.set_ylim(-1e-2, 1e-2)

In [ ]:
#xgrid.shape
xgrid

In [ ]:
power = PowerVsAngleSingleFreq(3000, angles, pattern_data)

In [ ]:
plt.plot(angles, 10 * np.log10(abs(power)))
plt.ylim(-50, -20)

In [ ]:
plt.plot(angles, np.angle(power))